### LSTM

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-I.csv', parse_dates=['Date'], index_col=['Date'])
data

,Close,ema20,ema50,rsi,macd,macd_signal,Price_Up
Date,,,,,,,
2015-02-11,219.184998,229.270971,250.640711,42.433898,-8.608123,-9.327306,1
2015-02-12,221.764008,228.556022,249.508291,43.901363,-8.210237,-9.103892,1
2015-02-13,235.427002,229.210401,248.956083,51.024304,-6.715013,-8.626116,1
2015-02-14,257.321014,231.887602,249.284120,59.826830,-3.720486,-7.644990,0
2015-02-15,234.824997,232.167354,248.717096,49.902257,-3.126502,-6.741292,0
...,...,...,...,...,...,...,...
2024-12-21,97224.726562,99069.601344,92289.965450,48.952697,1965.864891,3012.484494,0
2024-12-22,95104.937500,98692.014312,92400.356511,44.934433,1437.643624,2697.516320,0
2024-12-23,94686.242188,98310.512205,92489.999087,44.163328,974.011455,2352.815347,1


In [5]:
features = ['Close', 'ema20', 'ema50', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target].to_numpy()
y.shape, y

((3606, 1),
 array([[1],
        [1],
        [1],
        ...,
        [1],
        [0],
        [0]], shape=(3606, 1)))

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X) # Ya lo convierte en un array de numpy 
X_scaled.shape, X_scaled

((3606, 6),
 array([[-0.99983593, -0.99998561, -0.99963285, -0.22936958, -0.16634259,
         -0.18473173],
        [-0.99978724, -1.        , -0.99965728, -0.19458808, -0.16627684,
         -0.18469061],
        [-0.99952927, -0.99998683, -0.99966919, -0.02576196, -0.16602974,
         -0.18460268],
        ...,
        [ 0.78373741,  0.97353383,  0.99012755, -0.18837907, -0.00395957,
          0.25001379],
        [ 0.8590673 ,  0.9742344 ,  0.99536067,  0.00978761, -0.01322471,
          0.19719707],
        [ 0.85039487,  0.97398802,  1.        , -0.01290268, -0.02826776,
          0.15159296]], shape=(3606, 6)))

In [7]:
sequence_length = 30 # Ventana temporal de 30 dias 
X_sequences = []
y_sequences = []
for i in range(sequence_length, len(X_scaled)): # X_scaled tiene dimension de 1000, X_sequences tendra 1000 - 30(sequence_leght)
    X_sequences.append(X_scaled[i - sequence_length: i])
    y_sequences.append(y[i])
X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)
X_sequences.shape, y_sequences.shape


((3576, 30, 6), (3576, 1))

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_sequences, test_size=0.1, shuffle=False)

### Implementacion con Pytorch

In [9]:
# Convertimos los datos de Numpy a Tensores de Pytorch 
import torch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape 

(torch.Size([3218, 30, 6]),
 torch.Size([358, 30, 6]),
 torch.Size([3218, 1]),
 torch.Size([358, 1]))

In [10]:
# Creamos Datasets y Dataloaders de Pytorch
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor) # Instanciamos la clase TendorDataSet 
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size, shuffle=True) # Instanciamos la clase DataLoader
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

print(len(train_loader))
for batch_X, batch_y in train_loader:
    print("Batch X shape:", batch_X.shape)
    print("Batch y shape:", batch_y.shape)
      # Solo el primer batch
    break


101
Batch X shape: torch.Size([32, 30, 6])
Batch y shape: torch.Size([32, 1])


In [13]:
import torch
import torch.nn as nn 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)

class MiLSTM (nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        # Definimos 1 capa LSTM y un Full Connected 
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) #Ej [8, 64, 2, (batch_size, seq_length, hidden_size)]
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Estados ocultos y celdas en 0 (num_layers, batch_size, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # Ej [2, 32, 64]
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # Ej [2, 32, 64]

        # Pasa la entrada por la capa LSTM 
        out, _ = self.lstm(x, (h0,c0)) #LLamo a la instancia de lstm (llamo a su metodo __call__ (no necesita nombre)) (batch_size, seq_length, hidden_size) [32, 30, 64]

        #Seleccionamos la ultima salida de la secuencia
        out = out[:,-1,:] # (batch_size , hidden_size) [32,64]

        #Pasa por la fc
        out = self.fc(out) # (batch_size, output_size) Ej [32,1]

        return out

# Parametros del modelo 
input_size = X_train_tensor.shape[2] 
hidden_size = 64 
num_layers  = 2
output_size = 1 # Clasificacion 

model = MiLSTM(input_size, hidden_size, num_layers, output_size).to(device)
print(model) 

cuda
MiLSTM(
  (lstm): LSTM(6, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


In [17]:
# Definir la función de pérdida y el optimizador
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 50 
model.train()
print(f'Entrenado en {device}')

for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device) #[32, 30, 8]
        batch_y = batch_y.to(device) #[32, 1]

        outputs = model(batch_X).squeeze() # [batch_size, 1] => [batch_size] 

        loss = criterion(outputs, batch_y.squeeze())

        loss.backward()

        optimizer.step()

        optimizer.zero_grad()
    if (epoch + 1 ) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')





Entrenado en cuda
Epoch [10/50], Loss: 0.6959
Epoch [20/50], Loss: 0.7131
Epoch [30/50], Loss: 0.6778
Epoch [40/50], Loss: 0.7342
Epoch [50/50], Loss: 0.6159


In [15]:
model.eval()  # Modo de evaluación
with torch.no_grad():
    correct = 0
    total = 0
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        outputs = model(batch_X).squeeze()   # Logits
        probs = torch.sigmoid(outputs)       # Probabilidades
        predicted = torch.round(probs)       # Predicciones binarias (0 o 1)

        total += batch_y.size(0)
        correct += (predicted == batch_y.squeeze()).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')

Accuracy on test set: 49.72%
